In [1]:
import wmfdata as wmf
import pandas as pd
import numpy as np
import random

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import FuncFormatter, FixedLocator
from datetime import timedelta, datetime

from IPython.display import display_html, display, HTML, clear_output
import warnings

pd.options.display.max_columns = None
pd.options.display.max_rows = 250

In [2]:
spark_session = wmf.spark.get_active_session()

if type(spark_session) == type(None):
    spark_session = wmf.spark.create_custom_session(
        master="yarn",
        app_name='automod-reviews-per-day',
        spark_config={
            "spark.driver.memory": "4g",
            "spark.dynamicAllocation.maxExecutors": 64,
            "spark.executor.memory": "16g",
            "spark.executor.cores": 4,
            "spark.sql.shuffle.partitions": 256,
            "spark.driver.maxResultSize": "2g"
        }
    )

spark_session.sparkContext.setLogLevel("ERROR")

clear_output()

spark_session

In [109]:
# prints a string at center of the output, bold if needed
def pr_centered(content, bold=False):
    if bold:
        content = f"<b>{content}</b>"
    
    centered_html = f"<div style='text-align:center'>{content}</div>"
    
    display(HTML(centered_html))


# display dataframes horizontally with title for each
def display_h(frames, space=100):
    html = ""
    
    for key in frames.keys():
        html_df =f'<div>{key} {frames[key]._repr_html_()}</div>'
        html += html_df
        
    html = f"""
    <div style="display:flex; justify-content: space-evenly;">
    {html}
    </div>"""
    
    display_html(html, raw=True)

In [120]:
# applies cell color to a given nth percentile
def style_percentile(i, percentile='50th'):
    return ['background-color: Aquamarine' if i.name == percentile else '' for _ in i]

# return quatiles for a given series (dataframe and column name)
def quantiles(frame, col='risk', style_median=False, return_counts=True):
    
    quantile_values = [0.1, 0.25, 0.5, 0.7, 0.9, 0.99]
    qdict = {f"{int(q * 100)}th": frame[col].quantile(q) for q in quantile_values}
    
    df = pd.DataFrame(list(qdict.items()), columns=['percentile', col])    
    
    if return_counts:
        df['count'] = df[col].apply(lambda x: round(frame[frame[col] >= x].shape[0] / 30, 0))
    
    df[col] = round(df[col], 3)
    df['count'] = df['count'].astype(int)
    df.set_index('percentile', inplace=True)
    
    if style_median:
        df = df.style.apply(style_percentile, axis=1).format("{:.1f}")
        return df
    else:
        return df

In [5]:
# paths to pre-calculated revert risk scores
# generated by https://gitlab.wikimedia.org/repos/research/knowledge_integrity/-/blob/mnz/examples/examples/notebooks/revertrisk_example.ipynb
rr_scores_path = '/user/paragon/riskobservatory/revertrisk_20212022_anonymous_bot.parquet'

rr_scores = spark_session.read.parquet(rr_scores_path)
rr_scores.createOrReplaceTempView('rr_scores')

rr_scores.printSchema()

root
 |-- rev_id: long (nullable = true)
 |-- wiki_db: string (nullable = true)
 |-- rev_timestamp: string (nullable = true)
 |-- revision_is_identity_reverted: boolean (nullable = true)
 |-- revision_seconds_to_identity_revert: long (nullable = true)
 |-- page_id: long (nullable = true)
 |-- revision_revert_risk: float (nullable = true)
 |-- user_is_anonymous: boolean (nullable = true)
 |-- user_is_bot: boolean (nullable = true)



In [14]:
wiki_comp = pd.read_csv('https://raw.githubusercontent.com/wikimedia-research/wiki-comparison/main/data-collection/snapshots/Jan_2023.tsv', sep='\t')
top15_wps = wiki_comp[wiki_comp['project code'] == 'wikipedia'][:15]['database code'].values.tolist()
top15_wps_sql = wmf.utils.sql_tuple(top15_wps)
mwh_snapshot = '2023-12'

In [15]:
query = f"""
WITH
    base AS (
        SELECT
            wiki_db,
            revision_id,
            page_namespace_is_content,
            revision_parent_id,
            page_is_redirect,
            CASE
                WHEN SIZE(event_user_is_bot_by_historical) > 0 THEN TRUE
                ELSE FALSE
            END AS is_bot,
            CASE 
                WHEN ARRAY_CONTAINS(mwh.event_user_groups_historical, 'sysop') THEN TRUE
                ELSE FALSE
            END AS is_sysop,
            revision_is_identity_revert AS is_revert
        FROM
            wmf.mediawiki_history
        WHERE
            snapshot = '{mwh_snapshot}'
            AND wiki_db IN {top15_wps_sql}
            AND YEAR(event_timestamp) = 2022
    ),
    
    self_reverts AS (
        SELECT
            b.*,
            CASE
                WHEN b.event_user_text = mwh.event_user_text THEN TRUE
                ELSE FALSE
            END is_self_revert
        FROM
            base b
        JOIN
            wmf.mediawiki_history mwh
            ON mwh.revision_first_identity_reverting_revision_id = b.revision_id
                AND mwh.wiki_db = b.wiki_db
        WHERE
            snapshot = '{mwh_snapshot}'
            AND is_revert    
    ),
    
    edits AS (
        SELECT
            *,
            NULL AS is_self_revert
        FROM
            base
        WHERE
            NOT is_revert
        UNION ALL
        SELECT
            *
        FROM 
            self_reverts
    )

SELECT
    COUNT(DISTINCT revision_id) AS total_edits,
    COUNT(DISTINCT CASE WHEN page_namespace_is_content THEN revision_id ELSE NULL END) AS mainspace_edits,
    COUNT(DISTINCT CASE WHEN page_namespace_is_content AND revision_parent_id = 0 THEN revision_id ELSE NULL END) AS page_creations,
    COUNT(DISTINCT CASE WHEN page_namespace_is_content AND revision_parent_id = 0 AND page_is_redirect THEN revision_id ELSE NULL END) AS page_creations_redirects,
    COUNT(DISTINCT CASE WHEN page_namespace_is_content AND revision_parent_id = 0 AND NOT page_is_redirect THEN revision_id ELSE NULL END) AS page_creations_non_redirects,
    COUNT(DISTINCT CASE WHEN page_namespace_is_content AND is_bot THEN revision_id ELSE NULL END) AS mainspace_bot_edits,
    COUNT(DISTINCT CASE WHEN page_namespace_is_content AND is_sysop THEN revision_id ELSE NULL END) AS mainspace_sysop_edits,
    COUNT(DISTINCT CASE WHEN page_namespace_is_content AND is_revert THEN revision_id ELSE NULL END) AS mainspace_reverts,
    COUNT(DISTINCT CASE WHEN page_namespace_is_content AND is_self_revert THEN revision_id ELSE NULL END) AS mainspace_self_reverts,
FROM
    edits
"""

reviews_info = wmf.spark.run(query)

AnalysisException: cannot resolve '`revision_id`' given input columns: []; line 61 pos 19;
'Project ['COUNT(distinct 'revision_id) AS total_edits#0, 'COUNT(distinct CASE WHEN 'page_namespace_is_content THEN 'revision_id ELSE null END) AS mainspace_edits#1, 'COUNT(distinct CASE WHEN ('page_namespace_is_content AND ('revision_parent_id = 0)) THEN 'revision_id ELSE null END) AS page_creations#2, 'COUNT(distinct CASE WHEN (('page_namespace_is_content AND ('revision_parent_id = 0)) AND 'page_is_redirect) THEN 'revision_id ELSE null END) AS page_creations_redirects#3, 'COUNT(distinct CASE WHEN (('page_namespace_is_content AND ('revision_parent_id = 0)) AND NOT 'page_is_redirect) THEN 'revision_id ELSE null END) AS page_creations_non_redirects#4, 'COUNT(distinct CASE WHEN ('page_namespace_is_content AND 'is_bot) THEN 'revision_id ELSE null END) AS mainspace_bot_edits#5, 'COUNT(distinct CASE WHEN ('page_namespace_is_content AND 'is_sysop) THEN 'revision_id ELSE null END) AS mainspace_sysop_edi